# AWS EC2 Setup with MySQL Installation and Database Configuration


* Mounts Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


* Installing Required Python Packages: boto3 and paramiko


In [2]:
!pip install boto3 paramiko

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 38.0 MB/s eta 0:00:00


* AWS and Paramiko Library Importation


In [3]:
import boto3
import paramiko
import time
import subprocess


* Declare important variable

In [4]:
# AWS Credentials
aws_access_key = 'your aws_access_key'
aws_secret_key = 'your aws_secret_key'

# EC2 instance details
#you can change according to your need
region_name = 'ap-south-1'
instance_type = 't2.micro'
ami_id = 'ami-0287a05f0ef0e9d9a'  #default ami Ubuntu 20.04 LTS 64-bit
key_pair_name = 'ETL_KP'
security_group_name = 'ETL_SG'
tag_key = 'Name'
tag_value = 'Etl_Project_server'

* AWS EC2 Connection Setup


In [5]:
# Create object for Connect to EC2
ec2 = boto3.resource('ec2', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key, region_name=region_name)

# Create object for Connect an EC2 client
ec2_client = boto3.client('ec2', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key, region_name=region_name)

* Configure AWS EC2 Instance, Key Pair, Security Group, and Instance Launch Setup

In [13]:
# Create key pair
key_pair = ec2.create_key_pair(KeyName=key_pair_name)
with open(f'{key_pair_name}.pem', 'w') as key_file:
    key_file.write(key_pair.key_material)
print(f'Key pair "{key_pair_name}" created and saved to {key_pair_name}.pem')

# Create security group
security_group = ec2.create_security_group(
    GroupName=security_group_name,
    Description='Security group for MySQL, HTTP, SSH access',
    VpcId='vpc-051c6bf2ff976c0e8'  # Replace with your VPC ID
)

# Authorize inbound rules for SSH, HTTP, and MySQL
security_group.authorize_ingress(
    IpProtocol='tcp',
    FromPort=22,
    ToPort=22,
    CidrIp='0.0.0.0/0'
)

security_group.authorize_ingress(
    IpProtocol='tcp',
    FromPort=80,
    ToPort=80,
    CidrIp='0.0.0.0/0'
)

security_group.authorize_ingress(
    IpProtocol='tcp',
    FromPort=3306,
    ToPort=3306,
    CidrIp='0.0.0.0/0'
)

print(f'Security group "{security_group_name}" created with SSH, HTTP, and MySQL access')

# Launch EC2 instance
instances = ec2.create_instances(
    ImageId=ami_id,
    InstanceType=instance_type,
    MinCount=1,
    MaxCount=1,
    KeyName=key_pair_name,
    SecurityGroupIds=[security_group.id],
    TagSpecifications=[
                {
                    'ResourceType': 'instance',
                    'Tags': [
                        {'Key': tag_key, 'Value': tag_value},
                        # Add more tags as needed
                    ]
                }
            ]
)

instance = instances[0]

# Wait for the instance to be running
print(f'Launching EC2 instance with ID: {instance.id}')
instance.wait_until_running()
instance.reload()
print(f'EC2 instance running with public IP: {instance.public_ip_address}')


Key pair "ETL_KP" created and saved to ETL_KP.pem
Security group "ETL_SG" created with SSH, HTTP, and MySQL access
Launching EC2 instance with ID: i-002d393f2d417afaa
EC2 instance running with public IP: 13.201.85.179


* SSH into AWS EC2 Instance and Execute MySQL Setup Commands
* before connecting attach a iam role with s3 permission to the EC2 instance
* change the sql user password at 'your_sql_user_password'

In [18]:
# SSH into the instance (you might need to replace the key file path and username)
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

key_file_path = f'{key_pair_name}.pem'
ssh_client.connect(instance.public_ip_address, username='ubuntu', key_filename=key_file_path)

# Commands to install MySQL server and client, check version, secure installation, and check status
#change the sql user password at 'your_sql_user_password'
ec2_commands = [
    'sudo apt-get upgrade -y',
    'sudo apt-get install nginx -y',
    'sudo systemctl enable nginx',
    'sudo systemctl start nginx',
    'sudo systemctl status nginx',
    'sudo apt-get update',
    'sudo apt-get install -y mysql-server mysql-client',
    'sudo systemctl daemon-reload',
    'sudo systemctl enable mysql.service',
    'sudo systemctl start mysql.service',
    'sudo systemctl status mysql.service',
    'sudo mysql --version',
    'sudo mysql -u root -pyour_sql_user_password -e "ALTER USER \'root\'@\'localhost\' IDENTIFIED WITH mysql_native_password BY \'your_sql_user_password\';"',
    'sudo mysql -u root -pyour_sql_user_password -e "CREATE DATABASE imdb;"',
    'sudo mysql -u root -pyour_sql_user_password -e "CREATE USER \'etl\' IDENTIFIED BY \'your_sql_user_password\';"',
    'sudo mysql -u root -pyour_sql_user_password -e "GRANT ALL PRIVILEGES ON imdb.* TO \'etl\';"',
    'sudo mysql -u root -pyour_sql_user_password -e "FLUSH PRIVILEGES;"',
    'sudo systemctl restart mysql.service'

]

# Execute each command
for command in ec2_commands:
    stdin, stdout, stderr = ssh_client.exec_command(command)
    output = stdout.read().decode('utf-8')
    error = stderr.read().decode('utf-8')

    # Print the output and error messages
    print(f"Command Output for '{command}':\n{output}")
    print(f"Command Error for '{command}':\n{error}")

# Close the SSH connection
ssh_client.close()


Command Output for 'sudo apt-get upgrade -y':
Reading package lists...
Building dependency tree...
Reading state information...
Calculating upgrade...
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.

Command Error for 'sudo apt-get upgrade -y':

Command Output for 'sudo apt-get install nginx -y':
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  fontconfig-config fonts-dejavu-core libdeflate0 libfontconfig1 libgd3
  libjbig0 libjpeg-turbo8 libjpeg8 libnginx-mod-http-geoip2
  libnginx-mod-http-image-filter libnginx-mod-http-xslt-filter
  libnginx-mod-mail libnginx-mod-stream libnginx-mod-stream-geoip2 libtiff5
  libwebp7 libxpm4 nginx-common nginx-core
Suggested packages:
  libgd-tools fcgiwrap nginx-doc ssl-cert
The following NEW packages will be installed:
  fontconfig-config fonts-dejavu-core libdeflate0 libfontconfig1 libgd3
  libjbig0 libjpeg-turbo8 libjpeg8 libnginx-mod-http-geoip


# Delet existance EC2 Instance | security group | key-pair (if needed)

* Describe EC2 intance with name and id to identify

In [6]:
def describe_ec2_instances(aws_access_key, aws_secret_key, aws_region):
    try:
        # Describe instances
        response = ec2_client.describe_instances()

        # Extract instance details
        instances = []
        for reservation in response['Reservations']:
            for instance in reservation['Instances']:
                instance_id = instance['InstanceId']
                instance_name = get_instance_name(instance)
                instances.append({
                    'InstanceId': instance_id,
                    'InstanceName': instance_name,
                })

        return instances

    except Exception as e:
        print(f"Error describing EC2 instances: {e}")
        return []

def get_instance_name(instance):
    # Attempt to get the instance name from the 'Name' tag
    for tag in instance.get('Tags', []):
        if tag['Key'] == 'Name':
            return tag['Value']

    return None

instances = describe_ec2_instances(aws_access_key, aws_secret_key, region_name)
for instance in instances:
    print(f"Instance ID: {instance['InstanceId']}, Instance Name: {instance['InstanceName']}")



Instance ID: i-0235688ee90ba58ab, Instance Name: Etl_Project_server



* describe all the security group with id and name

In [7]:
# Create a Security Group object
sg_response = ec2_client.describe_security_groups()

* Define describe_security_groups Function

In [8]:
# Describe all security groups in the region
def describe_security_groups(aws_access_key, aws_secret_key, region_name):
    try:
        # Print details of each security group
        for group in sg_response['SecurityGroups']:
            print(f"Security Group ID: {group['GroupId']}, Security Group Name: {group['GroupName']}")

    except Exception as e:
        print(f"Error: {e}")


In [9]:
# Function calling
describe_security_groups(aws_access_key, aws_secret_key, region_name)

Security Group ID: sg-0dae590430548e22c, Security Group Name: default
Security Group ID: sg-0bfd1f5b91dd79582, Security Group Name: ETL_SG


* declaration of parameters that is used in deleting a EC2, SG and KP

* NOTE : Select below parameter carefully

In [10]:
#Select Instance id which we want to delete
instance_id = instances[0]['InstanceId']
print(instance_id)

#Select security_group_id which we want to delete
security_group_id = sg_response['SecurityGroups'][1]['GroupId']
print(security_group_id)

#Select or enter Keypair name which we want to delete
key_pair = key_pair_name
print(key_pair)

i-0235688ee90ba58ab
sg-0bfd1f5b91dd79582
ETL_KP


* Define  delete_ec2_instance Function

* Check all the parameter carefully before run the function

* NOTE : if Exception occure then it's a Dependancy issue some keypair or SG are connected with other resources. (ignore it)

In [11]:
# Delete specific EC2, SG and KP
def delete_ec2_instance(instance_id, security_group_id, key_pair_name, aws_access_key, aws_secret_key, aws_region):
    try:
        # Terminate the EC2 instance
        ec2_client.terminate_instances(InstanceIds=[instance_id])
        print(f"EC2 instance with ID '{instance_id}' terminated successfully.")

        # Wait for the instance to be terminated
        waiter = ec2_client.get_waiter('instance_terminated')
        waiter.wait(InstanceIds=[instance_id])
        print(f"EC2 instance with ID '{instance_id}' is terminated.")

        # Delete the key pair
        ec2_client.delete_key_pair(KeyName=key_pair)
        print(f"Key pair with name '{key_pair}' deleted successfully.")

        # Delete the security group
        ec2_client.delete_security_group(GroupId=security_group_id)
        print(f"Security group with ID '{security_group_id}' deleted successfully.")

    except Exception as e:
        print(f"Error deleting resources: {e}")


* call function delete_ec2_instance

In [12]:
#Function calling
delete_ec2_instance(instance_id, security_group_id, key_pair_name, aws_access_key, aws_secret_key, region_name)

EC2 instance with ID 'i-0235688ee90ba58ab' terminated successfully.
EC2 instance with ID 'i-0235688ee90ba58ab' is terminated.
Key pair with name 'ETL_KP' deleted successfully.
Security group with ID 'sg-0bfd1f5b91dd79582' deleted successfully.
